# Installings

In [ ]:
pip install requests opencv-python numpy pandas

: 

## Baixando as imagens

In [ ]:
import pandas as pd
import requests
import cv2
import unittest
from PIL import Image
from io import BytesIO
import os

: 

Criei uma função pra baixar imagens e salvar os arquivos localmente

In [ ]:
urls = [
    'https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_960_720.jpg',
    'https://cdn.pixabay.com/photo/2015/04/23/22/00/tree-736885_960_720.jpg',
    'https://cdn.pixabay.com/photo/2015/06/19/21/24/avenue-815297_960_720.jpg',
    'https://cdn.pixabay.com/photo/2013/10/02/23/03/mountains-190055_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/26/14/52/nuthatch-8722044_1280.jpg',
    'https://cdn.pixabay.com/photo/2014/12/16/22/25/sunset-570881_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/25/11/55/ai-generated-8719633_960_720.png',
    'https://cdn.pixabay.com/photo/2015/12/01/20/28/road-1072823_960_720.jpg',
    'https://cdn.pixabay.com/photo/2017/12/10/15/16/white-horse-3010129_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/28/05/07/bird-8724812_960_720.jpg'
]

: 

In [ ]:
def criar_diretorio_se_necessario(diretorio):
    """Cria um diretório se ele não existir."""
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
        print(f"Diretório '{diretorio}' criado.")
    else:
        print(f"Diretório '{diretorio}' já existe.")

def baixar_imagem(url, diretorio, nome_arquivo):
    """Baixa uma imagem da URL especificada e a salva em um diretório."""
    try:
        response = requests.get(url)
        response.raise_for_status() 
        imagem = Image.open(BytesIO(response.content))
        caminho_completo = os.path.join(diretorio, nome_arquivo)
        imagem.save(caminho_completo)
        print(f"Imagem salva em: {caminho_completo}")
        return True
    except requests.RequestException as e:
        print(f"Erro ao baixar a imagem de {url}: {e}")
        return False
    except IOError as e:
        print(f"Erro ao salvar a imagem em {caminho_completo}: {e}")
        return False

# Exemplo de uso:
diretorio = os.getcwd() + './images'
criar_diretorio_se_necessario(diretorio)
for i, url in enumerate(urls):
    baixar_imagem(url, diretorio, f"image_{i}.jpg")


: 

## Processamento

### Convertendo a escala de cinza

In [ ]:
def converter_para_cinza(imagem_path):
    imagem = cv2.imread(imagem_path)
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY) if imagem is not None else None


: 

### Redimensionamento de imagens

In [ ]:
def redimensionar(imagem, escala=0.5):
    altura = int(imagem.shape[0] * escala)
    largura = int(imagem.shape[1] * escala)
    dimensoes = (largura, altura)
    return cv2.resize(imagem, dimensoes, interpolation=cv2.INTER_AREA) if imagem is not None else None

: 

### Detectando bordas

In [ ]:
def detectar_bordas(imagem):
    return cv2.Canny(imagem, 100, 200) if imagem is not None else None

: 

### Aumento de contraste

In [ ]:
def aumentar_contraste(imagem):
    if imagem is None:
        return None
    lab = cv2.cvtColor(imagem, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge([cl, a, b])
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

: 

### Gaussian Blur

In [ ]:
def aplicar_gaussian_blur(imagem):
    return cv2.GaussianBlur(imagem, (5, 5), 0) if imagem is not None else None

: 

### Rotacionando

In [ ]:
def rotacionar_imagem(imagem, angulo=90):
    altura, largura = imagem.shape[:2]
    ponto_central = (largura//2, altura//2)
    matriz_rotacao = cv2.getRotationMatrix2D(ponto_central, angulo, 1.0)
    return cv2.warpAffine(imagem, matriz_rotacao, (largura, altura)) if imagem is not None else None

: 

## Testes e Visualização

In [ ]:
dados = []
image_paths = ['images/image_{}.jpg'.format(i) for i in range(len(urls))]
for url, path in zip(urls, image_paths):
  img_cinza = converter_para_cinza(path)
  img_redimensionada = redimensionar(img_cinza)
  img_bordas = detectar_bordas(img_redimensionada)
  img_original = cv2.imread(path)
  img_contraste = aumentar_contraste(img_original)
  img_blur = aplicar_gaussian_blur(img_contraste)
  img_rotacionada = rotacionar_imagem(img_original)
  dados.append([path, img_cinza, img_redimensionada, img_bordas, img_contraste, img_blur, img_rotacionada])

: 

In [ ]:
df = pd.DataFrame(dados, columns=['Path', 'Cinza', 'Redimensionada', 'Bordas', 'Contraste', 'Blur', 'Rotacionada'])

: 

### Testes

In [ ]:
class TesteProcessamento(unittest.TestCase):
    def test_redimensionamento(self):
        for _, row in df.iterrows():
            self.assertIsNotNone(row['Redimensionada'], "Imagem redimensionada está vazia")

: 

### Visualização

In [ ]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

df.head()

: 